In [2]:
import pandas as pd
import re
import multiprocessing

In [3]:
cta_train_gt = pd.read_csv('SOTAB CTA/sotab_v2_cta_training_set.csv')
cta_test_gt = pd.read_csv('SOTAB CTA/sotab_v2_cta_test_set.csv')

In [4]:
cta_train_gt

,table_name,column_index,label
0,Book_11x17.pt_September2020_CTA.json.gz,3,Date
1,Book_12min.com_September2020_CTA.json.gz,0,Book/name
2,Book_12min.com_September2020_CTA.json.gz,2,Language
3,Book_1carpetcleaning.co.uk_September2020_CTA.j...,7,Person/name
4,Book_1carpetcleaning.co.uk_September2020_CTA.j...,2,BookFormatType
...,...,...,...
116882,TVEpisode_zazangels.com_September2020_CTA.json.gz,3,CreativeWorkSeries
116883,TVEpisode_zebrahead.org_September2020_CTA.json.gz,0,TVEpisode/name
116884,TVEpisode_zebrahead.org_September2020_CTA.json.gz,3,CreativeWorkSeries
116885,TVEpisode_zoids-col.net_September2020_CTA.json.gz,1,TVEpisode/name


In [5]:
cta_test_gt

,table_name,column_index,label
0,Book_booksie.com_September2020_CTA.json.gz,1,Date
1,Book_efl-esl.com_September2020_CTA.json.gz,0,Book/name
2,Book_libreriasnobel.es_September2020_CTA.json.gz,3,Person/name
3,Book_unizin.org_September2020_CTA.json.gz,2,Date
4,Book_unizin.org_September2020_CTA.json.gz,4,Number
...,...,...,...
1846,Person_crsdenver.com_September2020_CTA.json.gz,3,faxNumber
1847,Restaurant_brighton.express_September2020_CTA....,0,Restaurant/name
1848,LocalBusiness_atlantademolition.services_Septe...,3,email
1849,LocalBusiness_atlantademolition.services_Septe...,4,openingHours


In [6]:
cta_test_gt["class"] = cta_test_gt["table_name"].apply(lambda x: x.split("_")[0])

In [7]:
cta_test_gt

,table_name,column_index,label,class
0,Book_booksie.com_September2020_CTA.json.gz,1,Date,Book
1,Book_efl-esl.com_September2020_CTA.json.gz,0,Book/name,Book
2,Book_libreriasnobel.es_September2020_CTA.json.gz,3,Person/name,Book
3,Book_unizin.org_September2020_CTA.json.gz,2,Date,Book
4,Book_unizin.org_September2020_CTA.json.gz,4,Number,Book
...,...,...,...,...
1846,Person_crsdenver.com_September2020_CTA.json.gz,3,faxNumber,Person
1847,Restaurant_brighton.express_September2020_CTA....,0,Restaurant/name,Restaurant
1848,LocalBusiness_atlantademolition.services_Septe...,3,email,LocalBusiness
1849,LocalBusiness_atlantademolition.services_Septe...,4,openingHours,LocalBusiness


In [8]:
gt = {'train':{}, 'test':{}}
for index, row in cta_train_gt.iterrows():
    if row["table_name"] not in gt['train']:
        gt['train'][row["table_name"]] = {}
    gt['train'][row["table_name"]][row["column_index"]] = row["label"]
    
for index, row in cta_test_gt.iterrows():
    if row["table_name"] not in gt['test']:
        gt['test'][row["table_name"]] = {}
    gt['test'][row["table_name"]][row["column_index"]] = row["label"]

In [9]:
#Simple Preprocessing

def clean_text(text):
        
    if(isinstance(text, dict)):
        text = ' '.join([ clean_text(v) for k, v in text.items()] )
    elif(isinstance(text, list)):
        text = map(clean_text, text)
        text = ' '.join(text)
        
    if pd.isnull(text):
        return ''
    
    #Remove excess whitespaces
    text = re.sub(' +', ' ', str(text)).strip()
    
    return text

In [10]:
# Prepare format of input datasets for LM models: table_id, [labels], data, label_ids

#ORIGINAL
""" 
def get_all_table_columns(file_name, index):
    
    #By column
    if file_name in cpa_train_gt["table_name"].tolist():
        path = 'SOTAB CPA/Train/'+file_name
    else:
        path = 'SOTAB CPA/Test/'+file_name
    
    df = pd.read_json(path, compression='gzip', lines=True)
        
    cleaned_rows = []
    
    #Main column

    cleaned_main = " ".join([" ".join(clean_text(row).split()[:20]) for row in df[0].tolist()[:5]]) #select 20 words
    
    cleaned_rows.append(cleaned_main)
    
    for row in df.iloc[:, index].tolist():
        cleaned = " ".join(clean_text(row).split()[:20]) #select 20 words
        if cleaned != "":
            cleaned_rows.append(cleaned)
    
    return " ".join(cleaned_rows[:5]) #select rows """

' \ndef get_all_table_columns(file_name, index):\n    \n    #By column\n    if file_name in cpa_train_gt["table_name"].tolist():\n        path = \'SOTAB CPA/Train/\'+file_name\n    else:\n        path = \'SOTAB CPA/Test/\'+file_name\n    \n    df = pd.read_json(path, compression=\'gzip\', lines=True)\n        \n    cleaned_rows = []\n    \n    #Main column\n\n    cleaned_main = " ".join([" ".join(clean_text(row).split()[:20]) for row in df[0].tolist()[:5]]) #select 20 words\n    \n    cleaned_rows.append(cleaned_main)\n    \n    for row in df.iloc[:, index].tolist():\n        cleaned = " ".join(clean_text(row).split()[:20]) #select 20 words\n        if cleaned != "":\n            cleaned_rows.append(cleaned)\n    \n    return " ".join(cleaned_rows[:5]) #select rows '

In [11]:
# Prepare format of input datasets for LM models: table_id, [labels], data, label_ids


#COLUMN TYPE MODIFIED
def get_all_table_columns(file_name, index):
    
    #By column
    if file_name in cta_train_gt["table_name"].tolist():
        path = 'SOTAB CTA/Train/'+file_name
    else:
        path = 'SOTAB CTA/Test/'+file_name
    
    df = pd.read_json(path, compression='gzip', lines=True)
        
    cleaned_rows = []
    
 
    for row in df.iloc[:, index].tolist():
        
        cleaned = " ".join(clean_text(row).split()[:20]) #select 20 words
        
        if cleaned != "":
            cleaned_rows.append(cleaned)
    
    
    return " ".join(cleaned_rows[:5]) #select rows

In [12]:
test_examples = []
for table in gt['test']:
    for column in gt['test'][table]:
        col_str = get_all_table_columns(table, column)
        test_examples.append([table, column, col_str, gt['test'][table][column], table.split("_")[0]])

In [13]:
test_examples

[['Book_booksie.com_September2020_CTA.json.gz',
  1,
  '2008-01-16 2015-01-22 2012-08-15 2014-08-18 2010-04-11',
  'Date',
  'Book'],
 ['Book_efl-esl.com_September2020_CTA.json.gz',
  0,
  "Children's Curriculum - 2B Three ESL Murder Mystery ESL Activities Book 1 ESL Bingo - Comparatives! Life STories - An ESL Discussion Activity",
  'Book/name',
  'Book'],
 ['Book_efl-esl.com_September2020_CTA.json.gz',
  4,
  'English English English English English',
  'Language',
  'Book'],
 ['Book_libreriasnobel.es_September2020_CTA.json.gz',
  3,
  'Centoira, Mar�a Elena;Fern�ndez, Lucia CRESPO, MANUEL J. CRESPO, MANUEL J. Cornford, Annie May, Meredith',
  'Person/name',
  'Book'],
 ['Book_unizin.org_September2020_CTA.json.gz',
  2,
  '2016-01-19 2017-06-01 2015-07-06 2018-03-24 2018-09-01',
  'Date',
  'Book'],
 ['Book_unizin.org_September2020_CTA.json.gz',
  4,
  '2016.0 2017.0 2015.0 2018.0 2018.0',
  'Number',
  'Book'],
 ['Event_blugrottonj.com_September2020_CTA.json.gz',
  4,
  'Blu Grotto 

In [14]:
cta_test_gt[cta_test_gt["table_name"] == "Test\Book_1jour-1jeu.com_September2020_CTA.json.gz"] 

,table_name,column_index,label,class


In [15]:
train_examples = []
for table in gt['train']:
    for column in gt['train'][table]:
        col_str = get_all_table_columns(table, column)
        train_examples.append([table, column, col_str, gt['train'][table][column], table.split("_")[0]])

In [16]:
len(train_examples)

116887

In [17]:
import pickle

In [18]:
file_name='SOTAB CTA/sotabv2-cta-train-column.pkl'
f = open(file_name,'wb')
pickle.dump(train_examples,f)
f.close()

In [19]:
file_name='SOTAB CTA/sotabv2-cta-test-column.pkl'
f = open(file_name,'wb')
pickle.dump(test_examples,f)
f.close()